In [87]:
import pandas as pd
import gensim
from collections import namedtuple
import numpy as np

In [36]:
recipes_df = pd.read_csv("clean_data.csv")

In [37]:
def clean_list(ingredient_list):
    ingredient_list = ingredient_list[2:-2]
    ingredient_list = ''.join(ingredient_list)
    ingredient_list = ingredient_list.split("', '")
    return ingredient_list

In [38]:
recipes_df["Clean_bag"]=recipes_df["Bag_Of_Ingredients"].map(clean_list)

In [128]:
bag_of_ingredients = recipes_df["Clean_bag"][0:13000]

In [129]:
' '.join(bag_of_ingredients[0])

'chicken acorn squash sage rosemary butter ground allspice red pepper ground black pepper loaf sturdy white bread apples gala pink lady extravirgin olive oil red onion apple cider vinegar white miso allpurpose flour butter dry white wine chicken broth white miso ground pepper'

In [130]:
bag_of_ingredients = [' '.join(i) for i in bag_of_ingredients]

In [131]:
docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i, text in enumerate(bag_of_ingredients):
    tags = [i]
    docs.append(analyzedDocument(text, tags))

In [132]:
docs[40]

AnalyzedDocument(words='green unripe mangoes shallots indian thai green chiles ginger matchsticks cloves garlic matchsticks coconut vinegar white vinegar coconut milk coconut oil black mustard seeds dried red chiles snapped curry leaves fried onions basmati rice', tags=[40])

In [133]:
model = gensim.models.doc2vec.Doc2Vec(docs,vector_size=50, min_count=4, epochs=40)

In [134]:
model.dv[0]

array([ 0.16909082, -0.03647247, -0.40935826, -0.015297  , -0.42651433,
       -0.22351599,  0.29626876, -0.06090831, -0.27130464, -0.39945838,
        0.3750459 , -0.03316819, -0.05699537, -0.1964929 , -0.2396947 ,
       -0.27734798,  0.2548483 , -0.07835688,  0.1499732 , -0.31903744,
       -0.04004226, -0.3082933 , -0.26603878,  0.17921123, -0.06540035,
        0.12912829, -0.14297013, -0.00471811,  0.36063254,  0.15574457,
        0.09559243, -0.00530355,  0.11485536, -0.25383255, -0.09766246,
        0.04231807,  0.14270818,  0.14914736,  0.19074996,  0.04778679,
        0.4766084 , -0.14234996,  0.21107112, -0.33469436, -0.17490275,
        0.22559212,  0.18341985, -0.4618018 , -0.03809474,  0.02620652],
      dtype=float32)

In [166]:
ingredients = ["cheese","onion","tortilla","lettuce","sausage","ham","tomato","tuna","pesto","salsa","mustard","cucumber","potato","sweet"]

In [167]:
av_in_vector = model.infer_vector(ingredients)

In [163]:
from numpy import dot
from numpy.linalg import norm

def similar_recipe(ingredients):
    cos_sim = []
    for i in range(0,len(model.dv)):
            cos_sim.append(np.dot(ingredients,model.dv[i])/(norm(ingredients)*norm(model.dv[i])))
    dis_array = np.array(cos_sim)
    return dis_array

In [164]:
def getListofRecipes(cos_sim,n):
    n_index = (-cos_sim).argsort()[:n]
    titles = []
    for i in n_index:
        titles.append(recipes_df.iloc[i,1:3])
    return titles

In [168]:
pd.DataFrame(getListofRecipes(similar_recipe(av_in_vector),10))

,Title,Ingredients
2834,Cumin-Chile Lamb Kebabs with Garlic Yogurt,"['1 garlic clove, grated', '1/2 teaspoon finel..."
774,Sheet-Pan Chicken Meatballs and Charred Broccoli,"['2/3 cup ketchup', '1/4 cup Worcestershire sa..."
10143,Polenta and Vegetables with Roasted Red Pepper...,"['1 cup uncooked instant polenta', '2/3 cup gr..."
4426,Dirty Fried Rice,"['2 tablespoons vegetable oil, divided', '4 ou..."
5337,Mahi Mahi Fish Tacos with Chipotle Slaw and Ro...,['2 cups chopped peeled and cored fresh pineap...
1152,Grilled Pork Ribs with Gochujang Barbecue Sauce,"['2/3 cup apple cider vinegar', '1/2 cup (pack..."
5039,Spiced Dark and Stormy,"['2 teaspoons allspice berries', '2 teaspoons ..."
5102,Portobello Frites,"['2 russet potatoes, scrubbed', '1 teaspoon co..."
11838,The Cabbage Soup Diet,"['1 head green cabbage, finely shredded or cho..."
5341,Minestrone Salad,"['12 ounces fingerling or baby red potatoes', ..."
